## Day 5: Hydrothermal Venture

In [ ]:
#!value --name sampleRaw
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2

`common.fsx` script contains the definition for the `Point` type, which is basically a wrapper for `int*int`, but also provides the `(+)` operator and the `Zero` value. All these allow for generating ranges for this type using range generation syntax. For example:

In [ ]:
#load "common.fsx"
[| Point (1,1) .. Point (1,1) .. Point(3,3) |] |> display
[| Point (5,1) .. Point (-1,1) .. Point(3,3) |] |> display


index Item 0 Item1 Item2 1 1 1 Item1 Item2 2 2 2 Item1 Item2 3 3

index Item 0 Item1 Item2 5 1 1 Item1 Item2 4 2 2 Item1 Item2 3 3

For parsing we will use the regular expressions type provider, which is not only about to validate and parse an input, but also to define the type for matches.

In [ ]:
#r "nuget:FSharp.Text.RegexProvider"
open FSharp.Text.RegexProvider
open FSharp.Text.RegexExtensions

type private LineFromToRegex = Regex< @"^\s*(?<FromX>\d+),(?<FromY>\d+)\s*->\s*(?<ToX>\d+),(?<ToY>\d+)\s*$" >

let parseLine line =
    match LineFromToRegex().TryTypedMatch(line) with
    | None -> failwith "unparsed"
    | Some match' ->
        Point (match'.FromX.AsInt, match'.FromY.AsInt), Point (match'.ToX.AsInt, match'.ToY.AsInt)

let parse (input : string) =
    input.Split("\n", StringSplitOptions.RemoveEmptyEntries)
    |> Array.map parseLine

Installed Packages FSharp.Text.RegexProvider, 2.1.0

For example:

In [ ]:
parseLine "6,4 -> 2,0" |> display
parseLine "0,0 -> 8,8\r" |> display

Item1,Item2
"{ Point (6, 4): Item: ( 6, 4 ) }","{ Point (2, 0): Item: ( 2, 0 ) }"


Item1,Item2
"{ Point (0, 0): Item: ( 0, 0 ) }","{ Point (8, 8): Item: ( 8, 8 ) }"


In [ ]:
#!share sampleRaw --from value
let sampleVents = parse sampleRaw
sampleVents

index Item1 Item2 0 Item ( 0, 9 ) Item ( 5, 9 ) 1 Item ( 8, 0 ) Item ( 0, 8 ) 2 Item ( 9, 4 ) Item ( 3, 4 ) 3 Item ( 2, 2 ) Item ( 2, 1 ) 4 Item ( 7, 0 ) Item ( 7, 4 ) 5 Item ( 6, 4 ) Item ( 2, 0 ) 6 Item ( 0, 9 ) Item ( 2, 9 ) 7 Item ( 3, 4 ) Item ( 1, 4 ) 8 Item ( 0, 0 ) Item ( 8, 8 ) 9 Item ( 5, 5 ) Item ( 8, 2 )

In part one we only need vertical and horizontal lines:

In [ ]:
let getHV vents = 
    vents
    |> Array.filter (fun (Point (x1, y1), Point (x2, y2)) -> x1 = x2 || y1 = y2)
let sampleVentsHV = getHV sampleVents
sampleVentsHV

index Item1 Item2 0 Item ( 0, 9 ) Item ( 5, 9 ) 1 Item ( 9, 4 ) Item ( 3, 4 ) 2 Item ( 2, 2 ) Item ( 2, 1 ) 3 Item ( 7, 0 ) Item ( 7, 4 ) 4 Item ( 0, 9 ) Item ( 2, 9 ) 5 Item ( 3, 4 ) Item ( 1, 4 )

The following function will analyze the direction from one point to the other and build a path between them using the range generation syntax

In [ ]:
let populateLine (p1, p2)  = 
    let (Point (x1, y1)) = p1
    let (Point (x2, y2)) = p2
    let dx = sign (x2 - x1)
    let dy = sign (y2 - y1)
    [|p1..Point (dx,dy)..p2|]

For example:

In [ ]:
populateLine (Point (1,2), Point (5,2)) |> display
populateLine (Point (1,2), Point (5,2)) |> display

index Item 0 Item1 Item2 1 2 1 Item1 Item2 2 2 2 Item1 Item2 3 2 3 Item1 Item2 4 2 4 Item1 Item2 5 2

index Item 0 Item1 Item2 1 2 1 Item1 Item2 2 2 2 Item1 Item2 3 2 3 Item1 Item2 4 2 4 Item1 Item2 5 2

In [ ]:
let overlappingPoints vents = 
    vents
    |> Array.collect populateLine
    |> Array.groupBy id
    |> Array.map (fun (point, samePoints) -> point, samePoints.Length)

### Part 1

In [ ]:
let private format (ar) = 
    let builder = StringBuilder()
    Printf.bprintf builder "<table>"
    for i in 0..(Array2D.length2 ar - 1) do
        Printf.bprintf builder "<tr>"
        for j in 0..(Array2D.length1 ar - 1) do
            let num = ar.[i,j]
            if (num = 0) then 
                Printf.bprintf builder "<td>.</td>"
            else Printf.bprintf builder $"<td>{num}</td>"
        Printf.bprintf builder "</tr>"
    Printf.bprintf builder "</table>"
    builder.ToString();

Formatter.Register<int[,]>((fun (ar) -> format ar), "text/html")

In [ ]:
let private matrix = 
    let m = Array2D.create 10 10 0
    overlappingPoints sampleVentsHV
    |> Array.iter (fun (Point (x, y), len) -> m.[y,x] <- len)
    m
matrix |> display
overlappingPoints sampleVentsHV |> Array.filter (fun (p, len) -> len > 1) |> Array.length

.,.,.,.,.,.,.,1,.,.
.,.,1,.,.,.,.,1,.,.
.,.,1,.,.,.,.,1,.,.
.,.,.,.,.,.,.,1,.,.
.,1,1,2,1,1,1,2,1,1
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
2,2,2,1,1,1,.,.,.,.


5

For the actual input:

In [ ]:
#!value --name inputRaw --from-file ./data

In [ ]:
#!share inputRaw --from value
let vents = parse inputRaw
let ventsHV = getHV vents
overlappingPoints ventsHV |> Array.filter (fun (p, len) -> len > 1) |> Array.length

6283

### Part 2

In [ ]:
let private matrix = 
    let m = Array2D.create 10 10 0
    overlappingPoints sampleVents
    |> Array.iter (fun (Point (x, y), len) -> m.[y,x] <- len)
    m
matrix |> display
overlappingPoints sampleVents |> Array.filter (fun (p, len) -> len > 1) |> Array.length

1,.,1,.,.,.,.,1,1,.
.,1,1,1,.,.,.,2,.,.
.,.,2,.,1,.,1,1,1,.
.,.,.,1,.,2,.,2,.,.
.,1,1,2,3,1,3,2,1,1
.,.,.,1,.,2,.,.,.,.
.,.,1,.,.,.,1,.,.,.
.,1,.,.,.,.,.,1,.,.
1,.,.,.,.,.,.,.,1,.
2,2,2,1,1,1,.,.,.,.


12

For the actual input:

In [ ]:
overlappingPoints vents |> Array.filter (fun (p, len) -> len > 1) |> Array.length

18864